In [1]:
import numpy as np
import scipy as sp
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
RT_demand = pd.read_csv("datasets/nrel118/RT_bus_demand.csv")
ST_demand = pd.read_csv("datasets/nrel118/ST_bus_demand.csv")

In [3]:
def interpolator(RT_data, dt = .25):
    old_len = len(RT_data)
    new_len = old_len * 4
    
    accum = np.zeros(old_len)
    
    for i in range(old_len):
        accum[i] = np.sum(RT_data[0:(i + 1)])
        
    t = np.arange(.5, (old_len + .5), 1)
    
    f = interp1d(t, RT_data, bounds_error = False, fill_value = "extrapolate")
    t_new = np.arange(.25, (old_len + .25), .25)
    
    new_data = f(t_new)
    
    for i in range(old_len):
        hr_sum = RT_data[i]
        quarter_sum = np.sum(new_data[(i * 4):((i + 1) * 4)]) * dt
        offset = hr_sum - quarter_sum
        new_data[(i * 4):((i + 1) * 4)] += offset * dt
        
    return new_data

In [5]:
RT_demand_vals = RT_demand.values[:, 2:]
ST_demand_vals = ST_demand.values[:, 2:]

In [6]:
def get_new_demands(demand):
    times = [f"{i}" for i in np.arange(.25, 768.25, .25)]
    col_names = ["Bus", "Region"]
    col_names += times
    
    demand_vals = demand.values[:, 2:]
    
    df = pd.DataFrame(columns = col_names)
    
    df["Bus"] = demand.Bus
    df["Region"] = demand.Region
    
    for i in tqdm(range(len(demand))):
        new_data = interpolator(demand_vals[i, :])
        df.iloc[i, 2:] = new_data
        
    return df

In [7]:
new_RT_df = get_new_demands(RT_demand)
new_ST_df = get_new_demands(ST_demand)

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [00:13<00:00,  9.07it/s]


In [144]:
#new_RT_df.to_csv("datasets/nrel118/RT_bus_demand_15min.csv", index = False)
#new_ST_df.to_csv("datasets/nrel118/ST_bus_demand_15min.csv", index = False)

In [9]:
wind_RT = pd.read_csv("datasets/nrel118/Wind/RT.csv")
wind_ST = pd.read_csv("datasets/nrel118/Wind/ST.csv")

solar_RT = pd.read_csv("datasets/nrel118/Solar/RT.csv")
solar_ST = pd.read_csv("datasets/nrel118/Solar/ST.csv")

In [16]:
def get_new_genr(genr):
    cols = genr.columns
    df = pd.DataFrame(columns = cols)
    
    df.Hour = np.arange(.25, 768.25, .25)
    genr_vals = genr.values[:, 1:]
    
    for i in range(len(genr_vals[0, :])):
        new_data = interpolator(genr_vals[:, i])
        new_data[new_data < 0] = 0
        df.iloc[:, i + 1] = new_data
    
    return df

In [17]:
new_wind_RT = get_new_genr(wind_RT)
new_wind_ST = get_new_genr(wind_ST)

new_solar_RT = get_new_genr(solar_RT)
new_solar_ST = get_new_genr(solar_ST)

In [19]:
#new_wind_RT.to_csv("datasets/nrel118/Wind/RT_15min.csv", index = False)
#new_wind_ST.to_csv("datasets/nrel118/Wind/ST_15min.csv", index = False)
#new_solar_RT.to_csv("datasets/nrel118/Solar/RT_15min.csv", index = False)
#new_solar_ST.to_csv("datasets/nrel118/Solar/ST_15min.csv", index = False)